### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
commit_id= '5e2942fa82aadeb74180d02ce30e0d8f363b67ea'

In [6]:
bucket = "aegovan-data"

In [7]:
# trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
# testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
# valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_max_neg.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_max_neg.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_max_neg.json".format(bucket)
trainfile_entity_only = "s3://{}/processed_dataset/train_unique_negative_entity_only.json".format(bucket)
testfile_entity_only= "s3://{}/processed_dataset/test_unique_negative_entity_only.json".format(bucket)
valfile_entity_only= "s3://{}/processed_dataset/val_unique_negative_entity_only.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_multiclass.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_multiclass.json".format(bucket)
valfile="s3://{}/processed_dataset/val_multiclass.json".format(bucket)

pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [9]:
pub_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "test" : testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}



In [10]:
notype_inputs = {
    "train" : trainfile_entity_only,
    "val" :valfile_entity_only,
    "test" : testfile_entity_only,
    "PRETRAINED_BIOBERT" : pretrained_bert
}



In [11]:
BertNetworkFactoryhyperparameters = {
    "dataset":"PpiDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    
    "earlystoppingpatience":20
}

In [12]:
BertNetworkFactoryhyperparameters_multiclass = {
    "dataset":"PpiMulticlassDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20
}

In [13]:
BertNoType_NetworkFactoryhyperparameters = {
    "dataset":"PpiNoInteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile_entity_only.split("/")[-1],
    "valfile":valfile_entity_only.split("/")[-1],
    "testfile":testfile_entity_only.split("/")[-1],
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "git_commit_id" : commit_id,
    "accumulation_steps": 8
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [15]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 5e2942fa82aadeb74180d02ce30e0d8f363b67ea
    Fix error message


In [16]:
base_name = "ppi-bert-untype"
hyperparameters =BertNoType_NetworkFactoryhyperparameters #BertNetworkFactoryhyperparameters_multiclass #BertNoType_NetworkFactoryhyperparameters 
inputs = notype_inputs #pub_inputs 


In [17]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
             'commit': hyperparameters['git_commit_id']
             }

In [18]:
hyperparameters

{'accumulation_steps': 8,
 'batchsize': '8',
 'dataset': 'PpiNoInteractionDatasetFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'git_commit_id': '5e2942fa82aadeb74180d02ce30e0d8f363b67ea',
 'learningrate': 1e-05,
 'log-level': 'INFO',
 'network': 'RelationExtractorBioBertFactory',
 'testfile': 'test_unique_negative_entity_only.json',
 'trainfile': 'train_unique_negative_entity_only.json',
 'valfile': 'val_unique_negative_entity_only.json'}

In [19]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'test': 's3://aegovan-data/processed_dataset/test_unique_negative_entity_only.json',
 'train': 's3://aegovan-data/processed_dataset/train_unique_negative_entity_only.json',
 'val': 's3://aegovan-data/processed_dataset/val_unique_negative_entity_only.json'}

In [20]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name =base_name)

In [ ]:
estimator.fit(inputs)

2019-11-05 08:14:35 Starting - Starting the training job...
2019-11-05 08:14:37 Starting - Launching requested ML instances...
2019-11-05 08:15:33 Starting - Preparing the instances for training......
2019-11-05 08:16:49 Downloading - Downloading input data
2019-11-05 08:16:49 Training - Downloading the training image.........
2019-11-05 08:18:47 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-11-05 08:18:48,731 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-11-05 08:18:48,775 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-11-05 08:18:48,775 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-11-05 08:18:49,095 sagemaker-containers INFO     Module main_train_bert does not provide a setup.py. 
Generating setup.py
2019-11-05 08:18:49,09

2019-11-05 08:19:13,110 - algorithms.transform_berttext_tokenise - INFO - Unknown tokens count 0
2019-11-05 08:19:13,110 - algorithms.transform_berttext_tokenise - INFO - Completed TransformBertTextTokenise
2019-11-05 08:19:13,110 - algorithms.transform_berttext_token_to_index - INFO - Transforming TransformBertTextTokenToIndex
2019-11-05 08:19:13,489 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertTextTokenToIndex
2019-11-05 08:19:13,511 - algorithms.transform_berttext_tokenise - INFO - Transforming TransformBertTextTokenise
2019-11-05 08:19:15,361 - algorithms.transform_berttext_tokenise - INFO - Unknown tokens count 0
2019-11-05 08:19:15,362 - algorithms.transform_berttext_tokenise - INFO - Completed TransformBertTextTokenise
2019-11-05 08:19:15,362 - algorithms.transform_berttext_token_to_index - INFO - Transforming TransformBertTextTokenToIndex
2019-11-05 08:19:15,410 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertText

2019-11-05 08:32:44,722 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_dbb5a5b8-216a-4ea9-a649-f81cd6816008_20191105_083244.csv: 
[[263  53]
 [ 21  84]]
2019-11-05 08:32:44,724 - algorithms.BertTrain - INFO - Validation set result details: 0.7854407713498623 
2019-11-05 08:32:44,724 - algorithms.BertTrain - INFO - Snapshotting because the current score 0.7854407713498623 is greater than 0.7588774341351661 
2019-11-05 08:32:44,724 - algorithms.ModelSnapshotCallback - INFO - Snappshotting model to /opt/ml/model/best_snaphsotmodel.pt
2019-11-05 08:32:45,361 - algorithms.BertTrain - INFO - Run    809     4      2015     8/403         2% 60.152567 21.210701       0.9272       0.7854
###score: train_loss### 60.15256743505597
###score: val_loss### 21.210701309144497
###score: train_fscore### 0.9272175488940424
###score: val_fscore### 0.7854407713498623
2019-11-05 08:35:21,349 - algorithms.BertTrain - INFO - Train set result details:


2019-11-05 08:51:27,848 - algorithms.BertTrain - INFO - Train set result details:
2019-11-05 08:51:27,853 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_264c053a-3b10-426c-834e-a5e0b75b754d_20191105_085127.csv: 
[[2376    8]
 [   5  834]]
2019-11-05 08:51:27,858 - algorithms.BertTrain - INFO - Train set result details: 0.994769140689129
2019-11-05 08:51:27,858 - algorithms.BertTrain - INFO - Validation set result details:
2019-11-05 08:51:32,812 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_0688e4ff-ed54-47f8-b372-e1235a7a391a_20191105_085132.csv: 
[[288  28]
 [ 30  75]]
2019-11-05 08:51:32,814 - algorithms.BertTrain - INFO - Validation set result details: 0.8148355981557874 
2019-11-05 08:51:32,814 - algorithms.BertTrain - INFO - Snapshotting because the current score 0.8148355981557874 is greater than 0.8092516807950891 
2019-11-05 08:51:32,814 - algorithms.ModelSn

2019-11-05 09:07:38,307 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_adaf469d-a653-46c6-84fb-fb63032e1e80_20191105_090738.csv: 
[[277  39]
 [ 25  80]]
2019-11-05 09:07:38,308 - algorithms.BertTrain - INFO - Validation set result details: 0.8053629218677762 
2019-11-05 09:07:38,308 - algorithms.BertTrain - INFO - Run   2902    17      7254     8/403         2% 2.884019 43.515136       0.9968       0.8054
###score: train_loss### 2.884019449353218
###score: val_loss### 43.51513606007211
###score: train_fscore### 0.9967822481532621
###score: val_fscore### 0.8053629218677762
2019-11-05 09:10:14,034 - algorithms.BertTrain - INFO - Train set result details:
2019-11-05 09:10:14,039 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_4ede91bf-a54e-4d32-867b-232b8292dd0d_20191105_091014.csv: 
[[2374   10]
 [   0  839]]
2019-11-05 09:10:14,043 - algorithms.BertTrain - INFO - Train 

2019-11-05 09:26:19,679 - algorithms.BertTrain - INFO - Train set result details:
2019-11-05 09:26:19,684 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_4d1bb5ba-2f37-47c7-9b18-967ed38f24b0_20191105_092619.csv: 
[[2379    5]
 [   0  839]]
2019-11-05 09:26:19,688 - algorithms.BertTrain - INFO - Train set result details: 0.9979896780603206
2019-11-05 09:26:19,689 - algorithms.BertTrain - INFO - Validation set result details:
2019-11-05 09:26:24,642 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_3d788081-5c7f-4a3c-8221-09e08fcdd4c7_20191105_092624.csv: 
[[286  30]
 [ 29  76]]
2019-11-05 09:26:24,643 - algorithms.BertTrain - INFO - Validation set result details: 0.8134383848701752 
2019-11-05 09:26:24,643 - algorithms.BertTrain - INFO - Run   4029    24     10075     8/403         2% 1.969661 42.927229       0.9980       0.8134
###score: train_loss### 1.969660870730877
##